In [281]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import statsmodels.formula.api as smf

In [282]:
sample=pd.read_csv("progresa_sample.csv")
print (sample)

       year  sex  indig  dist_sec   sc  grc  fam_n    min_dist    dist_cap  \
0        97  0.0    0.0     4.473  1.0  7.0      7   21.168384   21.168384   
1        98  0.0    0.0     4.473  1.0  8.0      7   21.168384   21.168384   
2        97  1.0    0.0     4.473  1.0  6.0      7   21.168384   21.168384   
3        98  1.0    0.0     4.473  1.0  7.0      7   21.168384   21.168384   
4        97  0.0    0.0     4.473  1.0  2.0      7   21.168384   21.168384   
...     ...  ...    ...       ...  ...  ...    ...         ...         ...   
77245    98  0.0    1.0     3.148  1.0  4.0      6  137.473203  172.770829   
77246    97  1.0    1.0     3.148  1.0  1.0      6  137.473203  172.770829   
77247    98  1.0    1.0     3.148  1.0  2.0      6  137.473203  172.770829   
77248    97  0.0    1.0     3.148  0.0  2.0      3  137.473203  172.770829   
77249    98  0.0    1.0     3.148  0.0  3.0      3  137.473203  172.770829   

        poor  ... hohedu  hohwag  welfare_index  hohsex  hohage

In [283]:
#sample.describe()

In [284]:
#1.1 Summary statistics 
#Not consider village, folnum and year
sample_drop=sample.drop(['year', 'village', 'folnum'], axis=1)
sample_drop.isnull().sum()
sample_drop=sample_drop.dropna()
sample_drop.shape
#sample_drop

(67122, 18)

In [285]:
#Find sample mean 
sample_mean=sample_drop.mean().reset_index(name='sample_mean')

#Find Sample Standard Deviation
sample_stdev=sample_drop.std().reset_index(name='sample_stdev')

#Merge Mean and Std Deviation
merged_stat=pd.DataFrame(pd.merge(sample_mean,sample_stdev,on=['index']))

#Sort alphabetically
merged_stat.sort_values(by='index')


,index,sample_mean,sample_stdev
13,age,11.141369,2.995374
7,dist_cap,148.747707,77.134043
2,dist_sec,2.346990,2.111281
5,fam_n,7.224666,2.329421
4,grc,4.089494,2.447608
14,grc97,3.762656,2.449568
12,hohage,44.262224,11.457772
8,hohedu,2.853908,2.655106
11,hohsex,0.927744,0.258914
9,hohwag,592.782484,791.378223


progresa coding scheme is based  on that localities were randomly selected to receive the program activities during the first 2 years (Summer 1998 to Summer 2000). The 314 out of 495 localities were coded as BASAL(treatment=1), and the rest of the localities were coded as 0(CONTROL, until 2000)

In [286]:
#1.2 Differences at baseline
sample_filter=sample.dropna()

#Filter for year 97 and Poor groups
sample_filter=sample_filter[(sample_filter.year==97)&(sample_filter.poor=='pobre')]
#sample_filter

t_value = []
p_value = []
stats_significant = []

sample_filter_edit = pd.DataFrame(pd.merge(sample_filter[sample_filter.progresa == 'basal'].mean().
                                                   reset_index(name='Average value (Treatment villages)'), 
                                                   sample_filter[sample_filter.progresa == '0'].mean().
                                                   reset_index(name='Average value (Control villages)'),on=['index']))

#Find t, p values 
for i in list(sample_filter_edit['index']):
          t_value.append(stats.ttest_ind(list(sample_filter[sample_filter.progresa == 'basal'][i]), list(sample_filter[sample_filter.progresa == '0'][i]), nan_policy='omit').statistic)
          p_value.append(stats.ttest_ind(sample_filter[sample_filter.progresa == 'basal'][i], sample_filter[sample_filter.progresa == '0'][i], nan_policy='omit').pvalue)
          if stats.ttest_ind(sample_filter[sample_filter.progresa == 'basal'][i], sample_filter[sample_filter.progresa == '0'][i], nan_policy='omit').pvalue < 0.05:
                stats_significant.append('TRUE')
          else: stats_significant.append('FALSE')
        
# Merging      
sample_filter_edit['Difference (Treat - Control)'] = t_value   
sample_filter_edit['p-value'] = p_value
sample_filter_edit['Statistically_Significant'] = stats_significant      
sample_filter_edit

,index,Average value (Treatment villages),Average value (Control villages),Difference (Treat - Control),p-value,Statistically_Significant
0,year,97.000000,97.000000,NaN,NaN,FALSE
1,sex,0.520268,0.506556,2.334621,1.956962e-02,TRUE
2,indig,0.316744,0.325306,-1.561670,1.183760e-01,FALSE
3,dist_sec,2.399648,2.400687,-0.041252,9.670949e-01,FALSE
4,sc,0.822697,0.815066,1.689261,9.117959e-02,FALSE
5,grc,3.709214,3.733653,-0.851002,3.947750e-01,FALSE
6,fam_n,7.281032,7.302939,-0.800187,4.236088e-01,FALSE
7,min_dist,107.395231,103.098348,8.766462,1.936661e-18,TRUE
8,dist_cap,151.801472,154.492095,-2.927983,3.414194e-03,TRUE
9,hohedu,2.726729,2.660982,2.224346,2.613239e-02,TRUE


Question 1.3 )Do you find any statistically significant differences between treatment and control villages as
baseline?
Answer: The statistically significant difference in welafre_index, hohwag, hohage hohedu, dist_cap, min_dist, and sex

In [287]:
(sample_filter_edit[sample_filter_edit.Statistically_Significant=='TRUE'])

,index,Average value (Treatment villages),Average value (Control villages),Difference (Treat - Control),p-value,Statistically_Significant
1,sex,0.520268,0.506556,2.334621,1.956962e-02,TRUE
7,min_dist,107.395231,103.098348,8.766462,1.936661e-18,TRUE
8,dist_cap,151.801472,154.492095,-2.927983,3.414194e-03,TRUE
9,hohedu,2.726729,2.660982,2.224346,2.613239e-02,TRUE
10,hohwag,550.207086,580.694452,-3.640301,2.727661e-04,TRUE
11,welfare_index,656.810476,661.593513,-3.559301,3.724043e-04,TRUE
13,hohage,43.631234,44.270717,-4.761979,1.925767e-06,TRUE
15,village,236.520636,289.078756,-30.069721,8.322183e-196,TRUE


Question 1.4. (4pt) Why does it matter if there are differences at baseline?
Answer: To make a comparison for treatment and control groups, we need to have the same baseline to meausre the magintude or amount of treatment effects. If there are differences in the group at baseline, we might not be able choose random sets.

Question 1.5. (4pt) What does this imply about how to measure the impact of the treatment?
Since there are variables that have p-value>0.5, they have a significant difference between the treatment and the control groups. Hence we would not be able to accurately measure the impact of treatment effect because of those.


In [288]:
#2.1 Before-after estimator (10pt)
#Compare the schooling rate of poor households in progresa villages before (i.e. 1997) and after (i.e. 1998) the program.

#Question 2.1) 1. (2pt) compute the estimator by just comparing the average schooling rates for these villages.
 #filtering for before (1997, poor and treatment group)
sample=sample.dropna()
sample_bfbefore=sample[(sample.year==97)&(sample.poor=='pobre')&(sample.progresa=='basal')]
before_mean=sample_bfbefore.groupby(['village'])['sc'].mean().reset_index(name='mean')
print (before_mean)
print ("average schooling rate by villages before",before_mean.mean())

sample_bfafter=sample[(sample.year==98)&(sample.poor=='pobre')&(sample.progresa=='basal')]
after_mean=sample_bfafter.groupby(['village'])['sc'].mean().reset_index(name='mean')
print (after_mean)
print ("average schooling rate by villages after",after_mean.mean())


print ("diff in before after estimator (98-97) =",after_mean['mean'].mean()- before_mean['mean'].mean())

     village      mean
0          1  0.751773
1          2  0.681818
2          4  0.916107
3          5  0.960000
4          6  0.962963
..       ...       ...
314      498  0.945946
315      499  0.777778
316      500  0.625000
317      501  0.758621
318      505  0.851852

[319 rows x 2 columns]
average schooling rate by villages before village    242.742947
mean         0.813757
dtype: float64
     village      mean
0          1  0.809917
1          2  0.714286
2          4  0.946058
3          5  0.976744
4          6  0.903846
..       ...       ...
314      498  0.838710
315      499  0.848485
316      500  0.583333
317      501  0.795918
318      505  0.866667

[319 rows x 2 columns]
average schooling rate by villages after village    242.742947
mean         0.833742
dtype: float64
diff in before after estimator (98-97) = 0.019984378946272252


In [289]:
##Question 2.1) 2. (3pt) now re-compute the estimator using linear regression, and individual schooling rates. Do not 
#include other regressors.

sample_bfbefore1=sample[(sample.year==97)&(sample.poor=='pobre')]
m=smf.ols(formula='sc ~ progresa', data=sample_bfbefore1).fit()
m.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                     sc   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                     2.854
Date:                Wed, 05 Feb 2020   Prob (F-statistic):             0.0912
Time:                        06:05:10   Log-Likelihood:                -14223.
No. Observations:               30738   AIC:                         2.845e+04
Df Residuals:                   30736   BIC:                         2.847e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept             0.8151      0.004    229.078      0.000       0.808       0.822
progresa[T.basal]     0.0076      0.005      1.689      0.091      -0.001       0.016
==============================================================================
Omnibus:                     7697.607   Durbin-Watson:                   1.701
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            14940.093
Skew:                          -1.664   Prob(JB):                         0.00
Kurtosis:                       3.769   Cond. No.                         3.01
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [290]:
sample_bfafter1=sample[(sample.year==98)&(sample.poor=='pobre')]
m1=smf.ols(formula='sc ~ progresa', data=sample_bfafter1).fit()
m1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                     sc   R-squared:                       0.003
Model:                            OLS   Adj. R-squared:                  0.002
Method:                 Least Squares   F-statistic:                     65.64
Date:                Wed, 05 Feb 2020   Prob (F-statistic):           5.66e-16
Time:                        06:05:10   Log-Likelihood:                -11171.
No. Observations:               26155   AIC:                         2.235e+04
Df Residuals:                   26153   BIC:                         2.236e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept             0.8109      0.004    217.190      0.000       0.804       0.818
progresa[T.basal]     0.0383      0.005      8.102      0.000       0.029       0.048
==============================================================================
Omnibus:                     7487.800   Durbin-Watson:                   1.734
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            15756.374
Skew:                          -1.796   Prob(JB):                         0.00
Kurtosis:                       4.245   Cond. No.                         3.02
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [291]:
##Question 2.1)3. (3pt) Finally, estimate a multiple regression model that includes other covariates
m2=smf.ols(formula='sc~progresa+sex+min_dist+dist_cap+hohedu+hohwag+welfare_index+hohage', data =sample_bfafter1 ).fit()
m2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                     sc   R-squared:                       0.029
Model:                            OLS   Adj. R-squared:                  0.029
Method:                 Least Squares   F-statistic:                     97.94
Date:                Wed, 05 Feb 2020   Prob (F-statistic):          2.09e-161
Time:                        06:05:10   Log-Likelihood:                -10818.
No. Observations:               26155   AIC:                         2.165e+04
Df Residuals:                   26146   BIC:                         2.173e+04
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept             0.8427      0.017     50.796      0.000       0.810       0.875
progresa[T.basal]     0.0339      0.005      7.228      0.000       0.025       0.043
sex                   0.0252      0.005      5.561      0.000       0.016       0.034
min_dist              0.0004   7.46e-05      5.499      0.000       0.000       0.001
dist_cap              0.0003   3.99e-05      6.312      0.000       0.000       0.000
hohedu                0.0117      0.001     11.727      0.000       0.010       0.014
hohwag            -1.738e-06   3.25e-06     -0.535      0.593    -8.1e-06    4.63e-06
welfare_index     -8.057e-05   2.05e-05     -3.935      0.000      -0.000   -4.04e-05
hohage               -0.0024      0.000    -10.447      0.000      -0.003      -0.002
==============================================================================
Omnibus:                     7076.576   Durbin-Watson:                   1.757
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            14348.680
Skew:                          -1.719   Prob(JB):                         0.00
Kurtosis:                       4.159   Cond. No.                     7.62e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 7.62e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

Question 2.1)4. (2pt) compare all the estimators. Are your estimates statistically signifcant? What do they suggest
about the efficacy of the progresa program?
Amongst the average, linear and mutiple regression estimator, the intercept for progresra is for linear 0.0383 and 0.0339 for multiple. The intercept has increased. Also the variables apart from progresra are statistically significant causing effect on the treatment results.

##Question 2.2)Cross-sectional estimator
1. (2pt) Begin by estimating the impact of Progresa by compring the average enrollment rate among
poor households in the treatment villages and the average enrollment rate among poor households
in the control villages. What do you find?

In [292]:
##Question 2.2)1.

cross_control=sample[(sample.year==98)&(sample.poor=='pobre')&(sample.progresa=='0')]
cross_control_mean=cross_control.groupby(['village'])['sc'].mean().reset_index(name='mean')
print (cross_control_mean)
print ("average schooling rate in control villages",cross_control_mean['mean'].mean())

cross_treat=sample[(sample.year==98)&(sample.poor=='pobre')&(sample.progresa=='basal')]
cross_treat_mean=cross_treat.groupby(['village'])['sc'].mean().reset_index(name='mean')
print (cross_treat_mean)
print ("average schooling rate in treatment villages",cross_treat_mean['mean'].mean())

print ("diff in cross estimator =",cross_treat_mean['mean'].mean()- cross_control_mean['mean'].mean())

     village      mean
0          3  0.909091
1         13  0.711111
2         16  0.821429
3         20  0.924242
4         22  0.650794
..       ...       ...
178      496  0.666667
179      497  0.755556
180      502  0.882353
181      503  0.800000
182      504  0.809524

[183 rows x 2 columns]
average schooling rate in control villages 0.8086240746137184
     village      mean
0          1  0.809917
1          2  0.714286
2          4  0.946058
3          5  0.976744
4          6  0.903846
..       ...       ...
314      498  0.838710
315      499  0.848485
316      500  0.583333
317      501  0.795918
318      505  0.866667

[319 rows x 2 columns]
average schooling rate in treatment villages 0.8337415033712263
diff in cross estimator = 0.025117428757507887


In [293]:
##Question 2.2)2.2. (3pt) Now repeat the estimator using simple regression.
m3=smf.ols(formula='sc~progresa', data=sample[(sample.year==98)&(sample.poor=='pobre')&(sample.progresa=='0')]).fit()
print (m3.summary())

m4=smf.ols(formula='sc~progresa', data=sample[(sample.year==98)&(sample.poor=='pobre')&(sample.progresa=='basal')]).fit()
print (m4.summary())

                            OLS Regression Results                            
Dep. Variable:                     sc   R-squared:                      -0.000
Model:                            OLS   Adj. R-squared:                 -0.000
Method:                 Least Squares   F-statistic:                      -inf
Date:                Wed, 05 Feb 2020   Prob (F-statistic):                nan
Time:                        06:05:11   Log-Likelihood:                -4750.4
No. Observations:                9869   AIC:                             9503.
Df Residuals:                    9868   BIC:                             9510.
Df Model:                           0                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.8109      0.004    205.724      0.0

In [294]:
##Question 2.2)3. (3pt) Third, use multiple regression to get the same estimate.
m5=smf.ols(formula='sc~progresa+sex+min_dist+dist_cap+hohedu+hohwag+welfare_index+hohage', 
           data=sample[(sample.year==98)&(sample.poor=='pobre')&(sample.progresa=='0')]).fit()
print (m5.summary())

m6=smf.ols(formula='sc~progresa+sex+min_dist+dist_cap+hohedu+hohwag+welfare_index+hohage',
           data=sample[(sample.year==98)&(sample.poor=='pobre')&(sample.progresa=='basal')]).fit()
print (m6.summary())

                            OLS Regression Results                            
Dep. Variable:                     sc   R-squared:                       0.025
Model:                            OLS   Adj. R-squared:                  0.024
Method:                 Least Squares   F-statistic:                     36.01
Date:                Wed, 05 Feb 2020   Prob (F-statistic):           4.53e-50
Time:                        06:05:11   Log-Likelihood:                -4625.8
No. Observations:                9869   AIC:                             9268.
Df Residuals:                    9861   BIC:                             9325.
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept         0.7765      0.028     28.201

##Question 2.2)4. (2pt) Finally, as above, compare your three estimators. What do you find? Are the effects statistically significant?
In the linear and multiple regression model for treatment groups, the intercept reduces from 0.8493 to 0.7765, and for control groups, the intercept reduces from  0.8109 to 0.7765. The standard error increases for treatment groups as we move from linear to multiple regressors. Variables hohwage become statistically insignificant in the multiple regression model for the treatment group.

##Question 2.3 Differences-in-differences estimator (30pt)
1. (6pt) Start with the simple table. However, DiD requires 4-way comparison. So compare the average
enrollment rate among poor households in the treatment villages and the average enrollment rate
among poor households in the control villages, both 1997 and 1998. What do you find?

In [295]:
###Question 2.3) 1.
beforecontrol=sample[(sample.year==97)&(sample.poor=='pobre')&(sample.progresa=='0')]
beforecontrolmean=beforecontrol.groupby(['village'])['sc'].mean().reset_index(name='mean')
print (beforecontrolmean)
print ("average schooling rate in control villages before treatment",beforecontrolmean['mean'].mean())

beforetreatment=sample[(sample.year==97)&(sample.poor=='pobre')&(sample.progresa=='basal')]
beforetreatmentmean=beforetreatment.groupby(['village'])['sc'].mean().reset_index(name='mean')
print (beforetreatmentmean)
print ("average schooling rate in treatment villages before treatment",beforetreatmentmean['mean'].mean())

aftercontrol=sample[(sample.year==98)&(sample.poor=='pobre')&(sample.progresa=='0')]
aftercontrolmean=aftercontrol.groupby(['village'])['sc'].mean().reset_index(name='mean')
print (aftercontrolmean)
print ("average schooling rate in control villages after treatment",aftercontrolmean['mean'].mean())

aftertreatment=sample[(sample.year==98)&(sample.poor=='pobre')&(sample.progresa=='basal')]
aftertreatmentmean=aftertreatment.groupby(['village'])['sc'].mean().reset_index(name='mean')
print (aftertreatmentmean)
print ("average schooling rate in treatment villages after treatment",aftertreatmentmean['mean'].mean())

diffcontrol= aftercontrolmean['mean'].mean() - beforecontrolmean['mean'].mean()
difftreatment =aftertreatmentmean['mean'].mean() - beforetreatmentmean['mean'].mean()
print ("diffcontrol", diffcontrol)
print ("difftreatment",difftreatment)
print  (" Difference in Difference of average enrollment is ", difftreatment- diffcontrol)

     village      mean
0          3  0.933333
1         13  0.742857
2         16  0.794118
3         20  0.959459
4         22  0.666667
..       ...       ...
178      496  0.666667
179      497  0.830189
180      502  0.911111
181      503  0.763889
182      504  0.800000

[183 rows x 2 columns]
average schooling rate in control villages before treatment 0.813671854981377
     village      mean
0          1  0.751773
1          2  0.681818
2          4  0.916107
3          5  0.960000
4          6  0.962963
..       ...       ...
314      498  0.945946
315      499  0.777778
316      500  0.625000
317      501  0.758621
318      505  0.851852

[319 rows x 2 columns]
average schooling rate in treatment villages before treatment 0.8137571244249541
     village      mean
0          3  0.909091
1         13  0.711111
2         16  0.821429
3         20  0.924242
4         22  0.650794
..       ...       ...
178      496  0.666667
179      497  0.755556
180      502  0.882353
181      50

In [296]:
###Question 2.3) 2. (10pt) Now repeat the estimator using simple regression.
#convert poor and time to binary 0 and 1
samplebinary=sample.dropna()
samplebinary.loc[samplebinary['poor'] == 'pobre', 'poor'] = 1
samplebinary.loc[samplebinary['poor'] == 'no pobre', 'poor'] = 0
samplebinary.loc[samplebinary.year == 97, 'time'] = 0
samplebinary.loc[samplebinary.year == 98, 'time'] = 1
samplebinary=samplebinary[samplebinary.poor==1]

m7=smf.ols(formula='sc~ progresa + time + progresa*time',
           data=samplebinary).fit()
m7.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                     sc   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     29.42
Date:                Wed, 05 Feb 2020   Prob (F-statistic):           5.32e-19
Time:                        06:05:13   Log-Likelihood:                -25412.
No. Observations:               56893   AIC:                         5.083e+04
Df Residuals:                   56889   BIC:                         5.087e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==========================================================================================
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                  0.8151      0.004    232.782      0.000       0.808       0.822
progresa[T.basal]          0.0076      0.004      1.717      0.086      -0.001       0.016
time                      -0.0041      0.005     -0.801      0.423      -0.014       0.006
progresa[T.basal]:time     0.0307      0.007      4.680      0.000       0.018       0.044
==============================================================================
Omnibus:                    15130.097   Durbin-Watson:                   1.392
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            30379.093
Skew:                          -1.722   Prob(JB):                         0.00
Kurtosis:                       3.976   Cond. No.                         7.63
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [297]:
###Question 2.3) 3. (8pt) And as above, use multiple regression to get the same estimate.
m8=smf.ols(formula='sc~ progresa + time + progresa*time+sex+min_dist+dist_cap+hohedu+hohwag+welfare_index+hohage',
           data=samplebinary).fit()
m8.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                     sc   R-squared:                       0.027
Model:                            OLS   Adj. R-squared:                  0.027
Method:                 Least Squares   F-statistic:                     157.3
Date:                Wed, 05 Feb 2020   Prob (F-statistic):               0.00
Time:                        06:05:13   Log-Likelihood:                -24680.
No. Observations:               56893   AIC:                         4.938e+04
Df Residuals:                   56882   BIC:                         4.948e+04
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
==========================================================================================
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                  0.8632      0.012     73.500      0.000       0.840       0.886
progresa[T.basal]          0.0036      0.004      0.818      0.413      -0.005       0.012
time                      -0.0060      0.005     -1.167      0.243      -0.016       0.004
progresa[T.basal]:time     0.0305      0.006      4.704      0.000       0.018       0.043
sex                        0.0291      0.003      9.288      0.000       0.023       0.035
min_dist                   0.0004   5.16e-05      7.246      0.000       0.000       0.000
dist_cap                   0.0003   2.77e-05      9.101      0.000       0.000       0.000
hohedu                     0.0117      0.001     16.902      0.000       0.010       0.013
hohwag                  9.068e-07   2.24e-06      0.404      0.686   -3.49e-06     5.3e-06
welfare_index             -0.0001   1.42e-05     -7.776      0.000      -0.000   -8.24e-05
hohage                    -0.0022      0.000    -14.440      0.000      -0.003      -0.002
==============================================================================
Omnibus:                    14326.170   Durbin-Watson:                   1.413
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            27820.570
Skew:                          -1.652   Prob(JB):                         0.00
Kurtosis:                       3.908   Cond. No.                     7.95e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 7.95e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

###Question 2.3) 4. (6pt) Finally, as above, compare your three estimators. What do you find? Are the effects statisti-
#cally signifcant?
The intercept significantly changes from 0.8151 to 0.8632 from linear to multiple regression model. All the variables included in this model are statistically significant. Accordign to the average means, the change in average schooling rate is  0.02503215931393077. The hohwag and year become statistically insignificant. Also, the progresra has a high p-value sugegsting this model is not the best fit.
The interaction coefficient of progresa and time is positive, indicating a positive impact of the program.

Question)2.4 Compare the estimators (20pt)
Now you have used three estimators to assess the effect of Progresa program.
1. (10pt) List the identifying assumptions (counterfactual assumptions) behind all three models. Which ones do you and more/less plausible?

In the before-after estimator, the assumption is that there could not have any observed difference in the schooling between year 97 and 98 if treatment was not introduced.

In the cross estimator, the assumption is that there could not have any observed difference in the schooling between treatment group(basal) and control group if treatment was not introduced.

In the Differnce in Difference Estimator counterfactual assumption is that the trends in treatment group and control group will be same if there was no treatment introduced in the group

Question 2.4) 2. (10pt) Compare the estimates of all three models. Do your analysis suggest that progresa program
had a positive impact on schooling rates?
Comparing the three estimators, the hohwag variable which is the head of household wage has been statistically the most insignificant for all. The DID estimator although has the highest intercept of the three, but has two insignificant variables, the hohwag and the time, making it not the best fit.
In the cross estimator, there are hohwag and welfare_index as statistically insignificant variables.
In before-after estimator, there is hohwag that is statistically insignificant.

The analysis suggest that progresa program had a positive impact on the schooling rates, since its respective coeffcients incremented in all cases.
Since we consider just the poor households, it makes sense why hohwag and welfare_index become insignificant in our analyis.